In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
from sqlalchemy import create_engine
from sqlalchemy.sql import text

import baikalfunctions as bfunc
import scheme_mar2023 as scheme
import mytools
import mysecure

In [ ]:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 100

In [ ]:
# pd.set_option('display.min_rows', 100)
%matplotlib inline
plt.style.use('dark_background')   # try another styles: 'classic'
plt.rcParams['figure.figsize'] = [15, 5]
#plt.subplots_adjust(top=1, left=0, right=1, bottom=0)

In [ ]:
pwd = 'G:\\1_Data1\\99_BaikalOct2024\\'
saveImgPath = pwd
saveDataPath = pwd
serverPath = 'C:\\xampp\\htdocs\\img\\'

In [ ]:
names=['Date', 'Time', 'DateUTC', 'TimeUTC', 'latitude', 'longitude', 'speed', 'course', 'solar_altitude', 'solar-azimuth',
       'temperature_air', 'pressure_air', 'relative-humidity_air', 'precipitation', 'solar_eye', 'solar_uv',
       'co2_picarro', 'co2_picarro_max', 'co2_picarro_min', 'ch4_picarro', 'ch4_picarro_max', 'ch4_picarro_min', 'h2o_picarro', 'co2_licor', 'h2o_licor', 'o2_water', 'o2_water_temperature',
       'ph', 'temperature_water', 'temperature_equ_1', 'temperature_equ_2', 'temperature_equ_3', 'temperature_equ_4', 'flow_water', 'flow_air', 'channel', 'flag',
       'fluo_nx_red', 'fluo_nx_green', 'fluo_nx_blue', 'fluo_kfa_red', 'fluo_kfa_green', 'fluo_kfa_blue', # 'comment',
       'water_flow_1', 'water_flow_2', 'water_flow_3', 'water_flow_4',
       'temperature_ds_1', 'temperature_ds_2', 'temperature_ds_3', 'temperature_ds_4', 'temperature_ds_5', 'temperature_ds_6', 'temperature_ds_7'
      ]

In [ ]:
df = pd.read_csv(pwd+'origin\\data.txt',
                 sep='\s+',
                 header=None,
                 names=names,
                 index_col=False,
                 parse_dates={'DateTime':['Date', 'Time']},
                 dayfirst=True,
                 na_values='-999999',
                 decimal=',',
                 encoding_errors='ignore',
                 #nrows=17820,
                )
# df['DateTime'] = pd.to_datetime([df['Date'], df['Time']])

df.info()

In [ ]:
df.tail(5)

In [ ]:
colweneed = ['DateTime', 'temperature_air', 'pressure_air', 'precipitation', 'solar_eye', 'solar_uv',
             'co2_picarro', 'ch4_picarro', 'h2o_picarro', 'flow_air', 'channel', 'flag',
             'fluo_nx_red', 'fluo_nx_green', 'fluo_nx_blue', 'fluo_kfa_red',
             'fluo_kfa_green', 'fluo_kfa_blue', 'water_flow_1', 'water_flow_2',
             'water_flow_3', 'water_flow_4', 'temperature_ds_1', 'temperature_ds_2', 'temperature_ds_3']

df.drop(axis='index', columns=[x for x in df.columns if x not in colweneed], inplace=True)
df.columns

In [ ]:
df.rename(columns=scheme.NAME_CONV_BAS, inplace=True)
df.sort_values(by='DateTime', inplace=True)
df.reset_index(inplace=True, drop=True)
df.head(20)

In [ ]:
df.describe()

In [ ]:
plt.rcParams['figure.figsize'] = [15, 15]
fig, axs = plt.subplots(8, 1)

axs[0].set_title('vCO2, ppm')
axs[0].plot(df.DateTime, df['vCO2'], '-', c='silver', linewidth=0.1)

axs[1].set_title('vCH4, ppm')
axs[1].plot(df.DateTime, df['vCH4'], '.', c='silver', markersize=1)

axs[2].set_title('Temp')
axs[2].plot(df.DateTime, df['TempEqu1'], 'r-')
axs[2].plot(df.DateTime, df['TempEqu2'], 'g-')
axs[2].plot(df.DateTime, df['TempEqu3'], 'b-')
axs[2].plot(df.DateTime, df['TempAir'], 'c-')

axs[3].set_title('WaterFlow')
axs[3].plot(df.DateTime, df['WaterFlowEqu1'], 'r-')
axs[3].plot(df.DateTime, df['WaterFlowEqu2'], 'g-')
axs[3].plot(df.DateTime, df['WaterFlowEqu3'], 'b-')

axs[4].set_title('Press')
axs[4].plot(df.DateTime, df['PressAir'], 'c-')
axs[5].set_title('Solar')
axs[5].plot(df.DateTime, df['LightLX'], 'w-')
axs[6].set_title('Fluo')
axs[6].plot(df.DateTime, df['FluoKfaBlu'], 'b.')
axs[6].plot(df.DateTime, df['FluoKfaGrn'], 'g.')
axs[6].plot(df.DateTime, df['FluoKfaRed'], 'r.')
axs[7].set_title('AirFlow')
axs[7].plot(df.DateTime, df['AirFlow'], 'b-')



### Correction data

In [ ]:
## gether/apply specific variables
### data corrections
# df['vCO2'] = df['vCO2'] * scheme.CO2PicarroScaut_cor[0] + scheme.CO2PicarroScaut_cor[1]
# df['vCH4'] = df['vCH4'] * scheme.CH4PicarroScaut_cor[0] + scheme.CH4PicarroScaut_cor[1]
# df['vCO2'] = df['CO2_dry']
# df['vCH4'] = df['CH4_dry']
df['LightLX'] = df['LightLX'] * scheme.solar_cor[0] + scheme.solar_cor[1]
df['LightUV'] = df['LightUV'] * scheme.solar_uv_cor[0] + scheme.solar_uv_cor[1]
df['PressAir'] = bfunc.pressConvert(df['PressAir'])
df['PressAir'] = df['PressAir'] * scheme.press_cor[0] + scheme.press_cor[1]

### servise data corrections
df['TempEqu1'] = df['TempEqu1'] * scheme.tempEqu1_cor[0] + scheme.tempEqu1_cor[1]
df['TempEqu2'] = df['TempEqu2'] * scheme.tempEqu2_cor[0] + scheme.tempEqu2_cor[1]
df['TempEqu3'] = df['TempEqu3'] * scheme.tempEqu3_cor[0] + scheme.tempEqu3_cor[1]

df['AirFlow'] = df['AirFlow'] * scheme.airflow_cor[0] + scheme.airflow_cor[1]
df['WaterFlowEqu1'] = df['WaterFlowEqu1'] * scheme.waterflowEqu1_cor[0] + scheme.waterflowEqu1_cor[1]
df['WaterFlowEqu2'] = df['WaterFlowEqu2'] * scheme.waterflowEqu2_cor[0] + scheme.waterflowEqu2_cor[1]
df['WaterFlowEqu3'] = df['WaterFlowEqu3'] * scheme.waterflowEqu3_cor[0] + scheme.waterflowEqu3_cor[1]

In [ ]:
v_state_list = list(scheme.CHANNEL_COLS.values())
v_state_list

In [ ]:
chnl = 'Channel'
if not chnl in df.columns:
    df[chnl] = 0
if 'V1_state' in df.columns:
    df.loc[df.V1_state == 1, chnl] = 1
    df.loc[df.V2_state == 1, chnl] = 2
    df.loc[df.V3_state == 1, chnl] = 3
    df.loc[df.V4_state == 1, chnl] = 4
    df.loc[df.V5_state == 1, chnl] = 5
    df.loc[df.V6_state == 1, chnl] = 6
df.drop(v_state_list, inplace=True, errors='ignore')

###  Remove bad data   
List of accidental cases to filter data (see notes)

In [ ]:
pd.options.display.max_rows=100

In [ ]:
df.loc[(df['DateTime'] > pd.to_datetime('04.10.2024 08:42', dayfirst=True)) & (df['DateTime'] < pd.to_datetime('04.10.2024 08:45', dayfirst=True)), ['Channel', 'vCO2']]

In [ ]:
filter_list = [{'date_start': '04.10.2024 08:43', 'date_stop': '04.10.2024 08:44', 'cols': ['Channel'], 'fill_with': 5},   # something wrong in the worls today
               {'date_start': '11.10.2024 00:47', 'date_stop': '11.10.2024 00:48', 'cols': ['Channel'], 'fill_with': 5},   # something wrong in the worls today
              ]

for cycle in filter_list:
    for col in cycle['cols']:
        date_start = pd.to_datetime(cycle['date_start'], dayfirst=True)
        date_stop = pd.to_datetime(cycle['date_stop'], dayfirst=True)
        df.loc[(df['DateTime'] > date_start) & (df['DateTime'] < date_stop), col] = cycle['fill_with']

In [ ]:
cols = ['FluoNxRed', 'FluoNxGrn', 'FluoNxBlu', 'FluoKfaRed', 'FluoKfaGrn', 'FluoKfaBlu']
for col in cols:
    df.loc[df[col] == 0, col] = np.nan

In [ ]:
cols = ['WaterFlowEqu1', 'WaterFlowEqu2', 'WaterFlowEqu3', 'WaterFlowEqu4', 'AirFlow']
for col in cols:
    df.loc[df[col] < 0, col] = np.nan

In [ ]:
df

In [ ]:
df = df.resample('20s', on='DateTime').mean()
df.reset_index(drop=False, inplace=True)

In [ ]:
df['Channel'] = df['Channel'].fillna(0)
df['event'] = 0
df.loc[df['Channel'] != df['Channel'].shift(1), ['event']] = 1
df['event'] = df['event'].cumsum()

In [ ]:
plt.rcParams['figure.figsize'] = [15, 5]
plt.scatter('DateTime', 'event', data=df, s=0.2, c='grey')

In [ ]:
df['valid'] = 1
df.loc[(df['event'] != df['event'].shift(-2)), ['valid']] = 0
df.loc[(df['event'] != df['event'].shift(10)), ['valid']] = 0
df['vCO2air'] = df['vCO2'][(df['Channel'] == 6) & (df['valid'] != 0)]
df['vCH4air'] = df['vCH4'][(df['Channel'] == 6) & (df['valid'] != 0)]

In [ ]:
df.index = df['DateTime']
for col in ['vCO2air', 'vCH4air',]:
    df[col] = df[col].rolling(3).mean()
    df[col] = df[col].interpolate(method='time')
df['pCO2air'] = df['vCO2air'] * df['PressAir']  # mkatm
df['pCH4air'] = df['vCH4air'] * df['PressAir']  # mkatm
for col in ['vCO2air', 'vCH4air',]:
    df[col] = df[col].rolling(200).mean()


In [ ]:
plt.scatter('DateTime', 'vCO2air', c='lime', s=10, data=df)
plt.scatter('DateTime', 'vCO2', c='grey', s=1, data=df, alpha=0.3)
#plt.xlim(pd.to_datetime('2024-10-04 00:00'), pd.to_datetime('2024-10-06 10:00'))

#plt.ylim(420, 460)

In [ ]:
plt.rcParams['figure.figsize'] = [15, 5]
fig, axs = plt.subplots(2, 1)

axs[0].set_title('CO2 air, ppm')
axs[0].set_ylim(400, 500)
axs[0].plot(df.DateTime, df['vCO2'], '-', c='silver')
axs[0].plot(df.DateTime, df['vCO2air'], 'r-')

axs[1].set_title('CH4 air, ppm')
axs[1].set_ylim(0, 10)
axs[1].plot(df.DateTime, df['vCH4'], '-', c='silver')
axs[1].plot(df.DateTime, df['vCH4air'], 'b-')
fig.savefig(saveImgPath+'cAir_vs_time.png', transparent=False)

In [ ]:
df['pCO2'] = df['vCO2'] * df['PressAir']
df['pCH4'] = df['vCH4'] * df['PressAir']

In [ ]:
df.columns

In [ ]:
df['DateSec'] = df['DateTime'].astype('int64') //10**9
df['dTSec'] = df['DateSec'] - df['DateSec'].shift(1)
dt = df['dTSec'] / 60   ## delta time, min

## Flux

In [ ]:
### find the moments where flux is valid to calculate
df['valid'] = (df['event'] == df['event'].shift(12)) & (df['event'] == df['event'].shift(-5))
df['vCO2chm'] = df['vCO2'][(df['Channel'] == 5) & (df['valid'] == 1)]
df['vCH4chm'] = df['vCH4'][(df['Channel'] == 5) & (df['valid'] == 1)]

df['fluxEvent'] = (df['Channel'] == 5) & (df['valid'] == 1)

In [ ]:
plt.rcParams['figure.figsize'] = [15, 5]
fig, axs = plt.subplots(1, 1)

axs.set_title('vCO2, ppm')
axs.scatter('DateTime', 'event', data=df.head(300), c='red', s=3)
axs.scatter('DateTime', 'valid', data=df.head(300), c='blue', s=1)

In [ ]:
plt.rcParams['figure.figsize'] = [15, 5]
fig, axs = plt.subplots(1, 1)

axs.set_title('vCO2, ppm')
axs.plot('DateTime', 'vCO2', '-', data=df.tail(1000), c='red')
axs.plot('DateTime', 'vCO2chm', '-', data=df.tail(1000), c='blue')

In [ ]:
ch_v = 'fluxEvent'
df['cCO2chm'] = df['vCO2chm'] * bfunc.pressConvert(df['PressAir']) * bfunc.getDensity(temp=0, press=1, gas='CO2') / 1000 # mg/l
df['cCH4chm'] = df['vCH4chm'] * bfunc.pressConvert(df['PressAir']) * bfunc.getDensity(temp=0, press=1, gas='CH4') # mkg/l
for col in ['cCO2chm', 'cCH4chm']:
    df[col] = df[col].rolling(3).mean()

In [ ]:
plt.scatter('DateTime', 'cCH4chm', data=df.tail(1000), c='red')

In [ ]:
deep = 2
df['CO2flux'] = (df['cCO2chm'].shift(-deep) - df['cCO2chm'].shift(deep)) / (df['DateSec'].shift(-deep) - df['DateSec'].shift(deep)) * 3600 / 0.81 * 320
df['CH4flux'] = (df['cCH4chm'].shift(-deep) - df['cCH4chm'].shift(deep)) / (df['DateSec'].shift(-deep) - df['DateSec'].shift(deep)) * 3600 / 0.81 * 320

In [ ]:
plt.rcParams['figure.figsize'] = [17, 5]
plt.plot('DateTime', 'CH4flux', 'ro', linewidth=1, alpha=0.2, data=df)
# ax.set_xlim(pd.to_datetime('02.10.2023 14:00', dayfirst=True), pd.to_datetime('08.10.2023 18:00', dayfirst=True))
plt.ylim(-50, 50)
plt.grid(alpha=0.2)

## RECOVERY !!!

#### recovery `CO2/CH4`, channel `1` (bottom in oct 2024)

In [ ]:
equ_vol = scheme.equ_walltube_param['equ_vol']  # equivalent equ volume, l
equ_cap = scheme.equ_walltube_param['equ_cap']  # equilibrator capacity
wtr_flow_min = scheme.equ_walltube_param['water_flow_min'] 

In [ ]:
t_wtr = df['TempEqu1']
t_air = t_wtr
wtr_flow = df['WaterFlowEqu1']
air_flow = df['AirFlow']

In [ ]:
### CO2 water
vGasEqu = df['vCO2'][(df[chnl] == 1) & (df[ch_v] == 1) & (wtr_flow > wtr_flow_min)]
solubility = bfunc.getSolubility(t_wtr, 'CO2')

density = bfunc.getDensity(t_air, df['PressAir'], 'CO2')
cGasAir = df['pCO2air'] / 1000000 * density
pGasEquAir = vGasEqu * df['PressAir']  # uatm
cGasEquAir = pGasEquAir * density / 1000000  # g/l

tau = equ_vol/(air_flow+wtr_flow*equ_cap*solubility/density)
eternal = (cGasEquAir-cGasEquAir.shift(1)*np.exp(-1*dt/tau))/(1-np.exp(-1*dt/tau))
cGasWtr = (eternal*(wtr_flow*equ_cap*solubility/density+air_flow)-air_flow*cGasAir)/(wtr_flow*equ_cap)

df['cCO2bot'] = cGasWtr * 1000     # mg/l
df['pCO2bot'] = cGasWtr * 1000000 / solubility  # mkatm

In [ ]:
### CH4 water
vGasEqu = df['vCH4'][(df[chnl] == 1) & (df[ch_v] == 1) & (wtr_flow > wtr_flow_min)]
solubility = bfunc.getSolubility(t_wtr, 'CH4')

density = bfunc.getDensity(t_air, df['PressAir'], 'CH4')
cGasAir = df['pCH4air'] / 1000000 * density
pGasEquAir = vGasEqu * df['PressAir']
cGasEquAir = pGasEquAir * density / 1000000  # g/l

tau = equ_vol/(air_flow+wtr_flow*equ_cap*solubility/density)
eternal = (cGasEquAir-cGasEquAir.shift(1)*np.exp(-1*dt/tau))/(1-np.exp(-1*dt/tau))
cGasWtr = (eternal*(wtr_flow*equ_cap*solubility/density+air_flow)-air_flow*cGasAir)/(wtr_flow*equ_cap)
df['cCH4bot'] = cGasWtr * 1000000000    # ng/l
df['pCH4bot'] = cGasWtr * 1000000 / solubility  # mkatm

#### recovery `CO2/CH4`, channel `2` (surface in mar 2024)

In [ ]:
t_wtr = df['TempEqu2']
t_air = t_wtr
wtr_flow = df['WaterFlowEqu2']

In [ ]:
### CO2 water
vGasEqu = df['vCO2'][(df[chnl] == 2) & (df[ch_v] == 1) & (wtr_flow > wtr_flow_min)]
solubility = bfunc.getSolubility(t_wtr, 'CO2')

density = bfunc.getDensity(t_air, df['PressAir'], 'CO2')
cGasAir = df['pCO2air'] / 1000000 * density
pGasEquAir = vGasEqu * df['PressAir']
cGasEquAir = pGasEquAir * density / 1000000  # g/l

tau = equ_vol/(air_flow+wtr_flow*equ_cap*solubility/density)
eternal = (cGasEquAir-cGasEquAir.shift(1)*np.exp(-1*dt/tau))/(1-np.exp(-1*dt/tau))
cGasWtr = (eternal*(wtr_flow*equ_cap*solubility/density+air_flow)-air_flow*cGasAir)/(wtr_flow*equ_cap)

df['cCO2sur'] = cGasWtr * 1000     # mg/l
df['pCO2sur'] = cGasWtr * 1000000 / solubility  # mkatm

In [ ]:
### CH4 water
vGasEqu = df['vCH4'][(df[chnl] == 2) & (df[ch_v] == 1) & (wtr_flow > wtr_flow_min)]
solubility = bfunc.getSolubility(t_wtr, 'CH4')

density = bfunc.getDensity(t_air, df['PressAir'], 'CH4')
cGasAir = df['pCH4air'] / 1000000 * density
pGasEquAir = vGasEqu * df['PressAir']
cGasEquAir = pGasEquAir * density / 1000000  # g/l

tau = equ_vol/(air_flow+wtr_flow*equ_cap*solubility/density)
eternal = (cGasEquAir-cGasEquAir.shift(1)*np.exp(-1*dt/tau))/(1-np.exp(-1*dt/tau))
cGasWtr = (eternal*(wtr_flow*equ_cap*solubility/density+air_flow)-air_flow*cGasAir)/(wtr_flow*equ_cap)
df['cCH4sur'] = cGasWtr * 1000000000    # ng/l
df['pCH4sur'] = cGasWtr * 1000000 / solubility  # mkatm

In [ ]:
## rolling average is applied inplace. Be cafelly, run this cell **ONCE**

for col in ['cCO2sur', 'cCO2bot', 'pCO2sur', 'pCO2bot', 'cCH4sur', 'pCH4sur', 'cCH4bot', 'pCH4bot']:
    df.loc[df[col] == np.inf , col] = np.nan 
    df.loc[df[col] == -np.inf , col] = np.nan
    df.loc[df[col] <= 0 , col] = np.nan
    df[col] = df[col].rolling(4, center=True).mean()

df['cCH4bot'].describe()

## Generate charts for server

In [ ]:
count_recent = 30000

plt.rcParams['figure.figsize'] = [15, 5]
fig, ax = plt.subplots()
## ax.set_xlim(pd.to_datetime('05.06.2023 18:00:00', dayfirst=True), pd.to_datetime('05.06.2023 19:00:00', dayfirst=True))
ax.set_title('CO2 water, mkatm')
ax.set_ylim(100, 600)
ax.plot(df.tail(count_recent).DateTime, df.tail(count_recent)['pCO2'], '-', c='#444', alpha=0.3)
ax.plot(df.tail(count_recent).DateTime, df.tail(count_recent)['pCO2sur'], 'g.', alpha=0.3)
ax.plot(df.tail(count_recent).DateTime, df.tail(count_recent)['pCO2bot'], 'r.', alpha=0.3)
ax.plot(df.tail(count_recent).DateTime, df.tail(count_recent)['pCO2air'], 'c.', alpha=0.1)

ax.grid(True, c='#555', alpha=0.8, linewidth=1, linestyle='--')
ax.xaxis.set_major_formatter(DateFormatter('%d.%m %H:%M'))

fig.savefig(serverPath+'pCO2wtr_vs_time.png', bbox_inches='tight')

In [ ]:
count_recent = 960

plt.rcParams['figure.figsize'] = [5, 5]
fig, ax = plt.subplots()
## ax.set_xlim(pd.to_datetime('05.06.2023 18:00:00', dayfirst=True), pd.to_datetime('05.06.2023 19:00:00', dayfirst=True))
ax.set_title('CO2 water, mkatm')
ax.plot(df.tail(count_recent).DateTime, df.tail(count_recent)['pCO2'], '-', c='#888', alpha=0.3)
ax.plot(df.tail(count_recent).DateTime, df.tail(count_recent)['pCO2sur'], 'g.', alpha=0.5, markersize=3)
ax.plot(df.tail(count_recent).DateTime, df.tail(count_recent)['pCO2bot'], 'r.', alpha=0.5, markersize=3)
ax.plot(df.tail(count_recent).DateTime, df.tail(count_recent)['pCO2air'], 'c.', alpha=0.3, markersize=1)

ax.grid(True, c='#555', alpha=0.8, linewidth=1, linestyle='--')
ax.xaxis.set_major_formatter(DateFormatter('%H:%M'))

fig.savefig(serverPath+'pCO2wtr_vs_time_3h.png', bbox_inches='tight')

In [ ]:
count_recent = 30000

plt.rcParams['figure.figsize'] = [15, 5]
fig, ax = plt.subplots()
ax.set_title('CH4 water, mkatm')
# ax.set_xlim(pd.to_datetime('06.06.2023 10:00:00', dayfirst=True), pd.to_datetime('06.06.2023 19:00:00', dayfirst=True))
# ax.set_ylim(0, 20)
ax.plot(df.tail(count_recent).DateTime, df.tail(count_recent)['pCH4'], '-', c='#444', alpha=0.3)
ax.plot(df.tail(count_recent).DateTime, df.tail(count_recent)['pCH4bot'], 'r.', alpha=0.2, markersize=3)
ax.plot(df.tail(count_recent).DateTime, df.tail(count_recent)['pCH4sur'], 'g.', alpha=0.2, markersize=3)
ax.plot(df.tail(count_recent).DateTime, df.tail(count_recent)['pCH4air'], 'c.', alpha=0.2, markersize=3)
ax.grid(True, c='#555', alpha=0.8, linewidth=1, linestyle='--')
ax.xaxis.set_major_formatter(DateFormatter('%d.%m %H:%M'))
fig.savefig(serverPath+'pCH4wtr_vs_time.png', bbox_inches='tight')

In [ ]:
count_recent = 960

fig, ax = plt.subplots()
ax.set_title('CH4 water, mkatm')
ax.plot(df.tail(count_recent).DateTime, df.tail(count_recent)['pCH4'], '-', c='#888', alpha=0.3)
ax.plot(df.tail(count_recent).DateTime, df.tail(count_recent)['pCH4bot'], 'r.', alpha=0.3, markersize = 3)
ax.plot(df.tail(count_recent).DateTime, df.tail(count_recent)['pCH4sur'], 'g.', alpha=0.3)
ax.plot(df.tail(count_recent).DateTime, df.tail(count_recent)['pCH4air'], 'c.', alpha=0.3)
ax.grid(True, c='#555', alpha=0.8, linewidth=1, linestyle='--')
ax.xaxis.set_major_formatter(DateFormatter('%H:%M'))
fig.savefig(serverPath+'pCH4wtr_vs_time_3h.png', bbox_inches='tight')

In [ ]:
count_recent = 30000

plt.rcParams['figure.figsize'] = [20, 5]
fig, ax = plt.subplots()
ax.set_title('temp water, grad C')
ax.plot(df.tail(count_recent).DateTime, df.tail(count_recent)['TempEqu1'], 'r.', alpha=0.3)
ax.plot(df.tail(count_recent).DateTime, df.tail(count_recent)['TempEqu2'], 'g.', alpha=0.3)
ax.grid(True, c='#555', alpha=0.8, linewidth=1, linestyle='--')
ax.xaxis.set_major_formatter(DateFormatter('%d.%m %H:%M'))
fig.savefig(serverPath+'tempWtr_vs_time.png', bbox_inches='tight')

In [ ]:
count_recent = 30000

plt.rcParams['figure.figsize'] = [20, 5]
fig, ax = plt.subplots()
ax.set_title('Water flow, l/min')
ax.plot(df.tail(count_recent).DateTime, df.tail(count_recent)['WaterFlowEqu1'], 'r.', alpha=0.3)
ax.plot(df.tail(count_recent).DateTime, df.tail(count_recent)['WaterFlowEqu2'], 'g.', alpha=0.3)
ax.grid(True, c='#555', alpha=0.8, linewidth=1, linestyle='--')
ax.xaxis.set_major_formatter(DateFormatter('%d.%m %H:%M'))
fig.savefig(serverPath+'waterFlow_vs_time.png', bbox_inches='tight')

In [ ]:
count_recent = 30000

plt.rcParams['figure.figsize'] = [20, 5]
fig, ax = plt.subplots()
ax.set_title('AirTemp, l/min')
ax.plot(df.tail(count_recent).DateTime, df.tail(count_recent)['TempAir'], 'r.', alpha=0.3)
ax.grid(True, c='#555', alpha=0.8, linewidth=1, linestyle='--')
ax.xaxis.set_major_formatter(DateFormatter('%d.%m %H:%M'))
fig.savefig(serverPath+'TempAir_vs_time.png', bbox_inches='tight')

In [ ]:
df.columns

In [ ]:
df['hour'] = df['DateTime'].dt.hour # + df['DateTime'].dt.minute / 60 # discomment for 
df['hour']

In [ ]:
cols = ['hour', 'DateSec', 'TempAir', 'PressAir', 'Precipitation', 'LightLX', 'LightUV',
        # 'METEO_LEVEL', 'METEO_PRESS', 'METEO_TEMP', 'METEO_HUM', 'METEO_SOLAR', 'METEO_WIND_DIR', 'METEO_WIND_SPEED', 'METEO_RAIN',
        'vCO2', 'vCH4', 'vH2O', 'AirFlow', 'Channel',
        'FluoNxRed', 'FluoNxGrn', 'FluoNxBlu', 'FluoKfaRed', 'FluoKfaGrn', 'FluoKfaBlu', 'WaterFlowEqu1',
        'WaterFlowEqu2', 'WaterFlowEqu3', 'WaterFlowEqu4',
        'TempEqu1', 'TempEqu2', 'TempEqu3',
        'cCO2chm', 'cCH4chm', 'CO2flux', 'CH4flux',
        'vCO2air', 'vCH4air',
        'pCO2air', 'pCH4air', 'cCO2sur', 'pCO2sur',
        'cCH4sur', 'pCH4sur', 'cCO2bot', 'pCO2bot', 'cCH4bot', 'pCH4bot',]
df.to_csv(f'{saveDataPath}origin.txt',
          columns=cols,
          sep='\t',
         )

In [ ]:
hourly = df.resample('60min', on='DateTime', offset='30min').mean()
hourly.index = hourly.index + pd.to_timedelta('30min')
hourly.to_csv(f'{saveDataPath}hourly.txt',
              columns=cols,
              sep='\t',
             )

In [ ]:
df

In [ ]:
func_list = ['mean', 'std']
cols_to_calc = {'cCO2sur': func_list,
                'cCO2bot': func_list,
               }

daily = df.groupby(by='hour', as_index=False).agg(cols_to_calc)
daily.to_csv(f'{saveDataPath}daily.txt',
             # columns=cols,
             sep='\t',
            )